In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pytube import YouTube


#url de vídeos do BernoulliTV pronto para busca, bastando acrescentar o código no final do link
URL_BUSCA_BPLAY = 'https://www.youtube.com/channel/UClsm8e0m-a-pLQTE31YmasQ/search?query='

with open('YoutubeAPI.txt', 'r') as f: # Lê a chave da API do Youtube
    your_api_key = f.read()

def dados_com_link(link): # Pega os dados de Likes, Dislikes, Visualizações e Data de postagem para o 'link'
    if link.count('=') == 1:
        id_of_video = link[link.find('=')+1:]
        url = 'https://www.youtube.com/watch?v=' + id_of_video
    if link.count('=') == 2:
        id_of_video = link[link.find('=')+1:link.rfind('=')]
        url = 'https://www.youtube.com/watch?v=' + id_of_video
    url_youtube_estatistica = f'https://www.googleapis.com/youtube/v3/videos?id={id_of_video}&key={your_api_key}&part=statistics'
    url_youtube_snippet = f'https://www.googleapis.com/youtube/v3/videos?id={id_of_video}&key={your_api_key}&part=snippet'
    json_estatistica = requests.get(url_youtube_estatistica).json()['items'][0]['statistics']
    json_snippet = requests.get(url_youtube_snippet).json()['items'][0]['snippet']
    visualizacoes = json_estatistica['viewCount']
    likes = json_estatistica['likeCount']
    dislikes = json_estatistica['dislikeCount']
    data_postagem = json_snippet['publishedAt'].split('T')[0]
    return {'Link':[url], 'Visualizações': [int(visualizacoes)], 'Likes': [int(likes)], 'Dislikes':[int(dislikes)], 'Postagem':[data_postagem]}

    
def link_com_codigo_BPlay(cod): # Usa a url de busca do BernoulliPlay para pegar o link de visualização do código 'cod'
    source = requests.get(URL_BUSCA_BPLAY + cod).text
    soup = BeautifulSoup(source, "html.parser")
    for link in soup.select('a[href^="/watch"]'):
        if cod.upper() in link.text.upper():
            parte_video = link.get('href')
            print(parte_video)
            return ('https://www.youtube.com' + parte_video)

        
# Pega os dados (inclusive o link) de todos os vídeos pelos códigos na lista_cod, acumula num DataFrame e salva o excel compilado
def script_dados_codigo(lista_cod):
    dados = pd.DataFrame()
    cont = 1
    tamanho = len(lista_cod)
    for codigo in lista_cod: 
        if cont == 1: print(f'Total: {tamanho}. Fazendo: ')
        print(f'{cont}, ', end = '')
        cont+=1
        dic = {'Código': [codigo.upper()]}
        time.sleep(1)
        try: # tenta a busca com símbolo zero ou símbolo vazio
            link_codigo = link_com_codigo_BPlay(codigo)
            dados_codigo = dados_com_link(link_codigo)
            dic.update(dados_codigo)
        except:
            link_codigo = link_com_codigo_BPlay(codigo.replace('0','Ø'))
            dados_codigo = dados_com_link(link_codigo)
            dic.update(dados_codigo)
        dados = pd.concat([dados, pd.DataFrame.from_dict(dic)], ignore_index = True, sort = True)
    dados.to_excel('_Dados_vídeos_gravados2.xlsx', index = False, encoding='utf16')

    
def baixar_videos(n, nome_excel):
    '''Baixa os n primeiros vídeos do excel gerado. (Por exemplo, ordenados por mais views no excel)
    Maior resolução em MP4 disponível, o nome do arquivo é o código do vídeo.'''
    
    dados = pd.read_excel(nome_excel)
    
    for linha in range(n):
        link = dados.iloc[linha,3]
        name = dados.iloc[linha,0]
        print(link, name)
        YouTube(link).streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(filename = name)
    
try:
    df_codigos = pd.read_csv('Videos gravados.csv')
    codigos = [cod.upper() for cod in df_codigos.iloc[:,0].tolist()]
    #script_dados_codigo(codigos) # Chama a função para rodar o script de fato.
except:
    print('CSV com os códigos dos vídeos gravados não está na pasta.')